# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [1]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver horse race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [14]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector from a normal distribution
mean = np.random.rand(N)
# Generate a random covariance matrix
A = np.random.rand(N, N)
cov = np.dot(A, A.transpose())  # To ensure the covariance matrix is positive semi-definite

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])


# Compute the vector of expected returns (mean returns) from df
scalefactor = 1
expected_return = ExpectedReturn(method='arithmetic', scalefactor=scalefactor)
q= expected_return.estimate(X=df, inplace=False)

# Compute the covariance matrix from df
covariance = Covariance(method='pearson')
P = covariance.estimate(X=df, inplace=False)


# Display the results
print("Vector of expected returns (q):")
print(q)
# 
print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
Asset_1      0.640808
Asset_2      0.179484
Asset_3      0.607434
Asset_4     -0.057847
Asset_5      0.508896
               ...   
Asset_96     0.236522
Asset_97     0.496411
Asset_98    -0.022446
Asset_99     0.409178
Asset_100    0.125338
Length: 100, dtype: float64

Covariance matrix (P):
             Asset_1    Asset_2    Asset_3    Asset_4    Asset_5    Asset_6  \
Asset_1    35.908528  27.769374  25.866829  26.649889  27.083634  27.782598   
Asset_2    27.769374  38.098514  27.050221  27.792044  27.550363  30.815317   
Asset_3    25.866829  27.050221  35.146148  25.473934  25.549242  29.346697   
Asset_4    26.649889  27.792044  25.473934  33.855364  24.548041  27.758137   
Asset_5    27.083634  27.550363  25.549242  24.548041  34.808593  27.860956   
...              ...        ...        ...        ...        ...        ...   
Asset_96   27.556890  28.294641  25.697731  26.649673  27.771625  28.849631   
Asset_97   25.725566  28.055820  25.926564

In [16]:
mean

array([0.84327227, 0.33279283, 0.57938234, 0.11061847, 0.44939613,
       0.77133017, 0.32663374, 0.94114959, 0.96320663, 0.34238742,
       0.88270503, 0.39901046, 0.62533041, 0.68319486, 0.38868207,
       0.15925195, 0.63539713, 0.28000947, 0.75229739, 0.4843771 ,
       0.82656712, 0.77662533, 0.57931831, 0.29804091, 0.17571095,
       0.96539813, 0.35819744, 0.64231055, 0.3118272 , 0.67064347,
       0.94650214, 0.0866464 , 0.01716606, 0.68550322, 0.05368673,
       0.36105651, 0.14852124, 0.84304714, 0.88549113, 0.88888332,
       0.29491922, 0.68191632, 0.80059527, 0.89854149, 0.34575876,
       0.39474191, 0.38205575, 0.85388519, 0.76876971, 0.9283753 ,
       0.69720655, 0.83450307, 0.66142521, 0.74347626, 0.71151703,
       0.70109329, 0.97700828, 0.5734034 , 0.67502591, 0.27583677,
       0.47840375, 0.30971579, 0.98508601, 0.49189496, 0.57443099,
       0.13686638, 0.62379821, 0.89114842, 0.94777433, 0.96170709,
       0.01719947, 0.19358678, 0.00770478, 0.5816621 , 0.84050

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [ ]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
#<your code here>

# Add box constraints (i.e., lower and upper bounds)
#<your code here>

# Add linear constraints
#<your code here>

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [ ]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
#<your code here>

Print and visualize the results

In [5]:
#<your code here>

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        #<your code here>

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            #<your code here>
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
#<your code here>

# Instantiate the MinVariance class
#<your code here>

# Prepare the optimization data and prepare the optimization problem
#<your code here>

# Solve the optimization problem and print the weights
#<your code here>